In [1]:
from pprint import pprint
from IPython.display import display

from fda_ode import *

In [2]:
eps, t, h, a, b = symbols(r'\varepsilon, t, h, alpha beta', real=True)
u, u1 = (f(t) for f in symbols('u, u1', cls=Function))

In [3]:
eq = eps*u.diff(t, 2) + (1 + eps**2)*u.diff(t) + (1 - eps**2)*u
print(latex(eq))

\varepsilon \frac{d^{2}}{d t^{2}} u{\left(t \right)} + \left(1 - \varepsilon^{2}\right) u{\left(t \right)} + \left(\varepsilon^{2} + 1\right) \frac{d}{d t} u{\left(t \right)}


Задача с пограничным слоем, страница 277

https://disk.yandex.ru/i/ng5ws4p2rYTcQA

In [4]:
s = dsolve(eq, u, ics={u.subs(t, 0): 0, u.subs(t, 1): 1})
print(s.args[1].expand().simplify())

(-exp(t/\varepsilon) + exp(t*(\varepsilon + 2)))*exp(-\varepsilon*t + \varepsilon - t + 1 - t/\varepsilon + 1/\varepsilon)/(-exp(1/\varepsilon) + exp(\varepsilon + 2))


In [5]:
print(s.args[1].expand().factor())

E*(exp(2*t)*exp(\varepsilon*t) - exp(t/\varepsilon))*exp(1/\varepsilon)*exp(\varepsilon)*exp(-t)*exp(-t/\varepsilon)*exp(-\varepsilon*t)/(-exp(1/\varepsilon) + exp(2)*exp(\varepsilon))


In [6]:
s.args[1].diff(t).subs(t, 0).expand().factor()

E*(\varepsilon**2 + 2*\varepsilon - 1)*exp(1/\varepsilon)*exp(\varepsilon)/(\varepsilon*(-exp(1/\varepsilon) + exp(2)*exp(\varepsilon)))

In [7]:
print(latex(s.args[1].diff(t).subs(t, 0).expand().factor()))

\frac{e \left(\varepsilon^{2} + 2 \varepsilon - 1\right) e^{\frac{1}{\varepsilon}} e^{\varepsilon}}{\varepsilon \left(- e^{\frac{1}{\varepsilon}} + e^{2} e^{\varepsilon}\right)}


In [8]:
print(ccode(s.args[1].diff(t).subs(t, 0).expand().factor()))

M_E*(pow(\varepsilon, 2) + 2*\varepsilon - 1)*exp(1.0/\varepsilon)*exp(\varepsilon)/(\varepsilon*(-exp(1.0/\varepsilon) + exp(2)*exp(\varepsilon)))


In [9]:
init((u, u1), t, h)

In [10]:
set_clip(8, 7, Rational(0, 1))

In [11]:
expand((T(u, 1)-T(u, -1))/(2*h))

h**6*Derivative(u(t), (t, 7))/5040 + h**4*Derivative(u(t), (t, 5))/120 + h**2*Derivative(u(t), (t, 3))/6 + Derivative(u(t), t)

In [12]:
def GaussLegendre2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y1/2 + y/2)
    return expand(k1)

In [13]:
def CrankNicolson2(f, y):
    y1 = Matrix([T(m, 1) for m in y])
    k1 = f(y)
    k2 = f(y1)
    return expand((k1 + k2)/2)

In [14]:
def RungeKutta4(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/2)
    k3 = f(y + h*k2/2)
    k4 = f(y + h*k3)
    return expand((k1 + 2*k2 + 2*k3 + k4)/6)

In [15]:
def DormandPrince5(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    return expand(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84)

def DormandPrince51(f, y):
    k1 = f(y)
    k2 = f(y + h*k1/5)
    k3 = f(y + h*(k1*3/40 + k2*9/40))
    k4 = f(y + h*(k1*44/45 - k2*56/15 + k3*32/9))
    k5 = f(y + h*(k1*19372/6561 - k2*25360/2187 + k3*64448/6561 - k4*212/729))
    k6 = f(y + h*(k1*9017/3168 - k2*355/33 + k3*46732/5247 + k4*49/176 - k5*5103/18656))
    k7 = f(y + h*(k1*35/384 + k3*500/1113 + k4*125/192 - k5*2187/6784 + k6*11/84))
    return expand(k1*5179/57600 + k3*7571/16695 + k4*393/640 - k5*92097/339200 + k6*187/2100 + k7/40)

In [16]:
def f(y):
    return Matrix([\
        clip_n(y[1]),\
        clip_n(-((1 + eps**2)*y[1] + (1 - eps**2)*y[0])/eps),\
    ])

## Gauss–Legendre method

In [17]:
set_clip(5, 4, Rational(0, 1))
r = GaussLegendre2(f, Matrix([u, u1]))

In [18]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-(u1_{t} - u_{tt})/2

h^2 =>


-(3*u1_{tt} - 2*u_{ttt})/12

h^3 =>


-(2*u1_{ttt} - u_{tttt})/24

- u_{1} + u_{t}
+h\big(
- \frac{u1_{t} - u_{tt}}{2}
\big)
+h^2\big(
- \frac{3 u1_{tt} - 2 u_{ttt}}{12}
\big)
+h^3\big(
- \frac{2 u1_{ttt} - u_{tttt}}{24}
\big)


In [19]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon*(u1_{t} - u_{t})/2 + u1_{tt}/2 + (u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


\varepsilon*(u1_{tt} - u_{tt})/4 + u1_{ttt}/6 + (u1_{tt} + u_{tt})/(4*\varepsilon)

h^3 =>


\varepsilon*(u1_{ttt} - u_{ttt})/12 + u1_{tttt}/24 + (u1_{ttt} + u_{ttt})/(12*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
\frac{\varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{u1_{tt}}{2} + \frac{u1_{t} + u_{t}}{2 \varepsilon}
\big)
+h^2\big(
\frac{\varepsilon \left(u1_{tt} - u_{tt}\right)}{4} + \frac{u1_{ttt}}{6} + \frac{u1_{tt} + u_{tt}}{4 \varepsilon}
\big)
+h^3\big(
\frac{\varepsilon \left(u1_{ttt} - u_{ttt}\right)}{12} + \frac{u1_{tttt}}{24} + \frac{u1_{ttt} + u_{ttt}}{12 \varepsilon}
\big)


In [20]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


\varepsilon**2*(u - u1)/12 - \varepsilon*u1/12 - u1/6 + u1/(12*\varepsilon) - (u + u1)/(12*\varepsilon**2)

h^3 =>


0

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{12} - \frac{\varepsilon u_{1}}{12} - \frac{u_{1}}{6} + \frac{u_{1}}{12 \varepsilon} - \frac{u + u_{1}}{12 \varepsilon^{2}}
\big)
+h^3\big(
0
\big)


In [21]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


-\varepsilon**3*(u - u1)/12 - \varepsilon**2*(u - 2*u1)/12 - \varepsilon*(u - 3*u1)/12 + u/6 + (u + 3*u1)/(12*\varepsilon) - (u + 2*u1)/(12*\varepsilon**2) + (u + u1)/(12*\varepsilon**3)

h^3 =>


0

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
0
\big)
+h^2\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{12} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{12} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{12} + \frac{u}{6} + \frac{u + 3 u_{1}}{12 \varepsilon} - \frac{u + 2 u_{1}}{12 \varepsilon^{2}} + \frac{u + u_{1}}{12 \varepsilon^{3}}
\big)
+h^3\big(
0
\big)


## Crank–Nicolson method

In [22]:
set_clip(5, 4, Rational(0, 1))
r = CrankNicolson2(f, Matrix([u, u1]))

In [23]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-(u1_{t} - u_{tt})/2

h^2 =>


-(3*u1_{tt} - 2*u_{ttt})/12

h^3 =>


-(2*u1_{ttt} - u_{tttt})/24

- u_{1} + u_{t}
+h\big(
- \frac{u1_{t} - u_{tt}}{2}
\big)
+h^2\big(
- \frac{3 u1_{tt} - 2 u_{ttt}}{12}
\big)
+h^3\big(
- \frac{2 u1_{ttt} - u_{tttt}}{24}
\big)


In [24]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon*(u1_{t} - u_{t})/2 + u1_{tt}/2 + (u1_{t} + u_{t})/(2*\varepsilon)

h^2 =>


\varepsilon*(u1_{tt} - u_{tt})/4 + u1_{ttt}/6 + (u1_{tt} + u_{tt})/(4*\varepsilon)

h^3 =>


\varepsilon*(u1_{ttt} - u_{ttt})/12 + u1_{tttt}/24 + (u1_{ttt} + u_{ttt})/(12*\varepsilon)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
\frac{\varepsilon \left(u1_{t} - u_{t}\right)}{2} + \frac{u1_{tt}}{2} + \frac{u1_{t} + u_{t}}{2 \varepsilon}
\big)
+h^2\big(
\frac{\varepsilon \left(u1_{tt} - u_{tt}\right)}{4} + \frac{u1_{ttt}}{6} + \frac{u1_{tt} + u_{tt}}{4 \varepsilon}
\big)
+h^3\big(
\frac{\varepsilon \left(u1_{ttt} - u_{ttt}\right)}{12} + \frac{u1_{tttt}}{24} + \frac{u1_{ttt} + u_{ttt}}{12 \varepsilon}
\big)


In [25]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


\varepsilon**2*(u - u1)/12 - \varepsilon*u1/12 - u1/6 + u1/(12*\varepsilon) - (u + u1)/(12*\varepsilon**2)

h^3 =>


0

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{12} - \frac{\varepsilon u_{1}}{12} - \frac{u_{1}}{6} + \frac{u_{1}}{12 \varepsilon} - \frac{u + u_{1}}{12 \varepsilon^{2}}
\big)
+h^3\big(
0
\big)


In [26]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


-\varepsilon**3*(u - u1)/12 - \varepsilon**2*(u - 2*u1)/12 - \varepsilon*(u - 3*u1)/12 + u/6 + (u + 3*u1)/(12*\varepsilon) - (u + 2*u1)/(12*\varepsilon**2) + (u + u1)/(12*\varepsilon**3)

h^3 =>


0

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
0
\big)
+h^2\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{12} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{12} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{12} + \frac{u}{6} + \frac{u + 3 u_{1}}{12 \varepsilon} - \frac{u + 2 u_{1}}{12 \varepsilon^{2}} + \frac{u + u_{1}}{12 \varepsilon^{3}}
\big)
+h^3\big(
0
\big)


## "original" Runge–Kutta method

In [27]:
set_clip(7, 6, Rational(0, 1))
r = RungeKutta4(f, Matrix([u, u1]))

In [28]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^4 =>


u_{ttttt}/120

h^5 =>


u_{tttttt}/720

- u_{1} + u_{t}
+h\big(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\big)
+h^2\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\big)
+h^3\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\big)
+h^4\big(
\frac{u_{ttttt}}{120}
\big)
+h^5\big(
\frac{u_{tttttt}}{720}
\big)


In [29]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^4 =>


u1_{ttttt}/120

h^5 =>


u1_{tttttt}/720

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\big)
+h^2\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\big)
+h^3\big(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \vare

In [30]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


-\varepsilon**4*(u - u1)/120 - \varepsilon**3*(2*u - 3*u1)/120 - \varepsilon**2*(2*u - 5*u1)/120 + \varepsilon*(2*u + 3*u1)/120 + u1/30 + (2*u - 3*u1)/(120*\varepsilon) + (2*u + 5*u1)/(120*\varepsilon**2) - (2*u + 3*u1)/(120*\varepsilon**3) + (u + u1)/(120*\varepsilon**4)

h^5 =>


-\varepsilon**5*(u - u1)/144 - \varepsilon**4*(3*u - 4*u1)/144 - \varepsilon**3*(u - 2*u1)/36 + \varepsilon**2*u1/18 + \varepsilon*(u + 7*u1)/144 + u/24 - (u - 7*u1)/(144*\varepsilon) - u1/(18*\varepsilon**2) + (u + 2*u1)/(36*\varepsilon**3) - (3*u + 4*u1)/(144*\varepsilon**4) + (u + u1)/(144*\varepsilon**5)

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{\varepsilon^{4} \left(u - u_{1}\right)}{120} - \frac{\varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{120} - \frac{\varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{120} + \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{120} + \frac{u_{1}}{30} + \frac{2 u - 3 u_{1}}{120 \varepsilon} + \frac{2 u + 5 u_{1}}{120 \varepsilon^{2}} - \frac{2 u + 3 u_{1}}{120 \varepsilon^{3}} + \frac{u + u_{1}}{120 \varepsilon^{4}}
\big)
+h^5\big(
- \frac{\varepsilon^{5} \left(u - u_{1}\right)}{144} - \frac{\varepsilon^{4} \cdot \left(3 u - 4 u_{1}\right)}{144} - \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{36} + \frac{\varepsilon^{2} u_{1}}{18} + \frac{\varepsilon \left(u + 7 u_{1}\right)}{144} + \frac{u}{24} - \frac{u - 7 u_{1}}{144 \varepsilon} - \frac{u_{1}}{18 \varepsilon^{2}} + \frac{u + 2 u_{1}}{36 \varepsilon^{3}} - \frac{3 u + 4 u_{1}}{144 \varepsilon^{4}} + \frac{u + u_{1}}{144 \varepsilon^{5}}
\big)


In [31]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


\varepsilon**5*(u - u1)/120 + \varepsilon**4*(3*u - 4*u1)/120 + \varepsilon**3*(u - 2*u1)/30 - \varepsilon**2*u1/15 - \varepsilon*(u + 7*u1)/120 - u/20 + (u - 7*u1)/(120*\varepsilon) + u1/(15*\varepsilon**2) - (u + 2*u1)/(30*\varepsilon**3) + (3*u + 4*u1)/(120*\varepsilon**4) - (u + u1)/(120*\varepsilon**5)

h^5 =>


\varepsilon**6*(u - u1)/144 + \varepsilon**5*(4*u - 5*u1)/144 + \varepsilon**4*(7*u - 12*u1)/144 + \varepsilon**3*(u - 4*u1)/36 - \varepsilon**2*(u + 15*u1)/144 - \varepsilon*(8*u + 7*u1)/144 - u1/18 - (8*u - 7*u1)/(144*\varepsilon) + (u - 15*u1)/(144*\varepsilon**2) + (u + 4*u1)/(36*\varepsilon**3) - (7*u + 12*u1)/(144*\varepsilon**4) + (4*u + 5*u1)/(144*\varepsilon**5) - (u + u1)/(144*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{\varepsilon^{5} \left(u - u_{1}\right)}{120} + \frac{\varepsilon^{4} \cdot \left(3 u - 4 u_{1}\right)}{120} + \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{30} - \frac{\varepsilon^{2} u_{1}}{15} - \frac{\varepsilon \left(u + 7 u_{1}\right)}{120} - \frac{u}{20} + \frac{u - 7 u_{1}}{120 \varepsilon} + \frac{u_{1}}{15 \varepsilon^{2}} - \frac{u + 2 u_{1}}{30 \varepsilon^{3}} + \frac{3 u + 4 u_{1}}{120 \varepsilon^{4}} - \frac{u + u_{1}}{120 \varepsilon^{5}}
\big)
+h^5\big(
\frac{\varepsilon^{6} \left(u - u_{1}\right)}{144} + \frac{\varepsilon^{5} \cdot \left(4 u - 5 u_{1}\right)}{144} + \frac{\varepsilon^{4} \cdot \left(7 u - 12 u_{1}\right)}{144} + \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{36} - \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{144} - \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{144} - \frac{u_{1}}{18} - \frac{8 u - 7 u

## Dormand–Prince method 5 order

In [32]:
set_clip(8, 7, Rational(0, 1))
r = DormandPrince5(f, Matrix([u, u1]))

In [33]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^5 =>


-\varepsilon**5*(u - u1)/600 - \varepsilon**4*(3*u - 4*u1)/600 - \varepsilon**3*(u - 2*u1)/150 + \varepsilon**2*u1/75 + \varepsilon*(u + 7*u1)/600 + (36*u + 5*u_{tttttt})/3600 - (u - 7*u1)/(600*\varepsilon) - u1/(75*\varepsilon**2) + (u + 2*u1)/(150*\varepsilon**3) - (3*u + 4*u1)/(600*\varepsilon**4) + (u + u1)/(600*\varepsilon**5)

h^6 =>


u_{ttttttt}/5040

- u_{1} + u_{t}
+h\big(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\big)
+h^2\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\big)
+h^3\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\big)
+h^4\big(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{120} + \frac{\varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{120} + \frac{\varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{120} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{120} - \frac{4 u_{1} - u_{ttttt}}{120} - \frac{2 u - 3 u_{1}}{120 \varepsilon} - \frac{2 u + 5 u_{1}}{120 \varepsilon^{2}} + 

In [34]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^5 =>


\varepsilon**6*(u - u1)/600 + \varepsilon**5*(4*u - 5*u1)/600 + \varepsilon**4*(7*u - 12*u1)/600 + \varepsilon**3*(u - 4*u1)/150 - \varepsilon**2*(u + 15*u1)/600 - \varepsilon*(8*u + 7*u1)/600 - (48*u1 - 5*u1_{tttttt})/3600 - (8*u - 7*u1)/(600*\varepsilon) + (u - 15*u1)/(600*\varepsilon**2) + (u + 4*u1)/(150*\varepsilon**3) - (7*u + 12*u1)/(600*\varepsilon**4) + (4*u + 5*u1)/(600*\varepsilon**5) - (u + u1)/(600*\varepsilon**6)

h^6 =>


u1_{ttttttt}/5040

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\big)
+h^2\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\big)
+h^3\big(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \vare

In [35]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^5 =>


-\varepsilon**5*(u - u1)/3600 - \varepsilon**4*(3*u - 4*u1)/3600 - \varepsilon**3*(u - 2*u1)/900 + \varepsilon**2*u1/450 + \varepsilon*(u + 7*u1)/3600 + u/600 - (u - 7*u1)/(3600*\varepsilon) - u1/(450*\varepsilon**2) + (u + 2*u1)/(900*\varepsilon**3) - (3*u + 4*u1)/(3600*\varepsilon**4) + (u + u1)/(3600*\varepsilon**5)

h^6 =>


-\varepsilon**6*(u - u1)/2100 - \varepsilon**5*(4*u - 5*u1)/2100 - \varepsilon**4*(7*u - 12*u1)/2100 - \varepsilon**3*(u - 4*u1)/525 + \varepsilon**2*(u + 15*u1)/2100 + \varepsilon*(8*u + 7*u1)/2100 + 2*u1/525 + (8*u - 7*u1)/(2100*\varepsilon) - (u - 15*u1)/(2100*\varepsilon**2) - (u + 4*u1)/(525*\varepsilon**3) + (7*u + 12*u1)/(2100*\varepsilon**4) - (4*u + 5*u1)/(2100*\varepsilon**5) + (u + u1)/(2100*\varepsilon**6)

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
- \frac{\varepsilon^{5} \left(u - u_{1}\right)}{3600} - \frac{\varepsilon^{4} \cdot \left(3 u - 4 u_{1}\right)}{3600} - \frac{\varepsilon^{3} \left(u - 2 u_{1}\right)}{900} + \frac{\varepsilon^{2} u_{1}}{450} + \frac{\varepsilon \left(u + 7 u_{1}\right)}{3600} + \frac{u}{600} - \frac{u - 7 u_{1}}{3600 \varepsilon} - \frac{u_{1}}{450 \varepsilon^{2}} + \frac{u + 2 u_{1}}{900 \varepsilon^{3}} - \frac{3 u + 4 u_{1}}{3600 \varepsilon^{4}} + \frac{u + u_{1}}{3600 \varepsilon^{5}}
\big)
+h^6\big(
- \frac{\varepsilon^{6} \left(u - u_{1}\right)}{2100} - \frac{\varepsilon^{5} \cdot \left(4 u - 5 u_{1}\right)}{2100} - \frac{\varepsilon^{4} \cdot \left(7 u - 12 u_{1}\right)}{2100} - \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{525} + \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{2100} + \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{2100} + \frac{2 u_{1}}{525} + \frac{8 u - 7 u_{1}}{2100 \varepsil

In [36]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^5 =>


\varepsilon**6*(u - u1)/3600 + \varepsilon**5*(4*u - 5*u1)/3600 + \varepsilon**4*(7*u - 12*u1)/3600 + \varepsilon**3*(u - 4*u1)/900 - \varepsilon**2*(u + 15*u1)/3600 - \varepsilon*(8*u + 7*u1)/3600 - u1/450 - (8*u - 7*u1)/(3600*\varepsilon) + (u - 15*u1)/(3600*\varepsilon**2) + (u + 4*u1)/(900*\varepsilon**3) - (7*u + 12*u1)/(3600*\varepsilon**4) + (4*u + 5*u1)/(3600*\varepsilon**5) - (u + u1)/(3600*\varepsilon**6)

h^6 =>


\varepsilon**7*(u - u1)/2100 + \varepsilon**6*(5*u - 6*u1)/2100 + \varepsilon**5*(11*u - 17*u1)/2100 + \varepsilon**4*(11*u - 28*u1)/2100 + \varepsilon**3*(3*u - 31*u1)/2100 - \varepsilon**2*(9*u + 22*u1)/2100 - \varepsilon*(7*u + 15*u1)/2100 - u/150 + (7*u - 15*u1)/(2100*\varepsilon) - (9*u - 22*u1)/(2100*\varepsilon**2) - (3*u + 31*u1)/(2100*\varepsilon**3) + (11*u + 28*u1)/(2100*\varepsilon**4) - (11*u + 17*u1)/(2100*\varepsilon**5) + (5*u + 6*u1)/(2100*\varepsilon**6) - (u + u1)/(2100*\varepsilon**7)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
0
\big)
+h^5\big(
\frac{\varepsilon^{6} \left(u - u_{1}\right)}{3600} + \frac{\varepsilon^{5} \cdot \left(4 u - 5 u_{1}\right)}{3600} + \frac{\varepsilon^{4} \cdot \left(7 u - 12 u_{1}\right)}{3600} + \frac{\varepsilon^{3} \left(u - 4 u_{1}\right)}{900} - \frac{\varepsilon^{2} \left(u + 15 u_{1}\right)}{3600} - \frac{\varepsilon \left(8 u + 7 u_{1}\right)}{3600} - \frac{u_{1}}{450} - \frac{8 u - 7 u_{1}}{3600 \varepsilon} + \frac{u - 15 u_{1}}{3600 \varepsilon^{2}} + \frac{u + 4 u_{1}}{900 \varepsilon^{3}} - \frac{7 u + 12 u_{1}}{3600 \varepsilon^{4}} + \frac{4 u + 5 u_{1}}{3600 \varepsilon^{5}} - \frac{u + u_{1}}{3600 \varepsilon^{6}}
\big)
+h^6\big(
\frac{\varepsilon^{7} \left(u - u_{1}\right)}{2100} + \frac{\varepsilon^{6} \cdot \left(5 u - 6 u_{1}\right)}{2100} + \frac{\varepsilon^{5} \cdot \left(11 u - 17 u_{1}\right)}{2100} + \frac{\varepsilo

## Dormand–Prince method 4 order

In [37]:
set_clip(7, 6, Rational(0, 1))
r = DormandPrince51(f, Matrix([u, u1]))

In [38]:
F1 = clip((T(u, 1)-T(u, 0))/h - T(r[0], 0))
prn(F1, eps)
prnlatex(F1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


-\varepsilon*(u - u1)/2 + u_{tt}/2 + (u + u1)/(2*\varepsilon)

h^2 =>


...

h^4 =>


1097*\varepsilon**4*(u - u1)/120000 + 1097*\varepsilon**3*(2*u - 3*u1)/120000 + 1097*\varepsilon**2*(2*u - 5*u1)/120000 - 1097*\varepsilon*(2*u + 3*u1)/120000 - (1097*u1 - 250*u_{ttttt})/30000 - 1097*(2*u - 3*u1)/(120000*\varepsilon) - 1097*(2*u + 5*u1)/(120000*\varepsilon**2) + 1097*(2*u + 3*u1)/(120000*\varepsilon**3) - 1097*(u + u1)/(120000*\varepsilon**4)

h^5 =>


-161*\varepsilon**5*(u - u1)/120000 - 161*\varepsilon**4*(3*u - 4*u1)/120000 - 161*\varepsilon**3*(u - 2*u1)/30000 + 161*\varepsilon**2*u1/15000 + 161*\varepsilon*(u + 7*u1)/120000 + (1449*u + 250*u_{tttttt})/180000 - 161*(u - 7*u1)/(120000*\varepsilon) - 161*u1/(15000*\varepsilon**2) + 161*(u + 2*u1)/(30000*\varepsilon**3) - 161*(3*u + 4*u1)/(120000*\varepsilon**4) + 161*(u + u1)/(120000*\varepsilon**5)

- u_{1} + u_{t}
+h\big(
- \frac{\varepsilon \left(u - u_{1}\right)}{2} + \frac{u_{tt}}{2} + \frac{u + u_{1}}{2 \varepsilon}
\big)
+h^2\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{6} - \frac{\varepsilon u_{1}}{6} - \frac{2 u_{1} - u_{ttt}}{6} + \frac{u_{1}}{6 \varepsilon} - \frac{u + u_{1}}{6 \varepsilon^{2}}
\big)
+h^3\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{24} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{24} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{24} + \frac{2 u + u_{tttt}}{24} + \frac{u + 3 u_{1}}{24 \varepsilon} - \frac{u + 2 u_{1}}{24 \varepsilon^{2}} + \frac{u + u_{1}}{24 \varepsilon^{3}}
\big)
+h^4\big(
\frac{1097 \varepsilon^{4} \left(u - u_{1}\right)}{120000} + \frac{1097 \varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{120000} + \frac{1097 \varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{120000} - \frac{1097 \varepsilon \left(2 u + 3 u_{1}\right)}{120000} - \frac{1097 u_{1} - 250 u_{ttttt}}{30000} - \frac{1097 \cdot \left(2 u - 3 u_{1}\right)

In [39]:
F2 = clip((T(u1, 1)-T(u1, 0))/h - T(r[1], 0))
prn(F2, eps)
prnlatex(F2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


\varepsilon**2*(u - u1)/2 - \varepsilon*u1/2 - (2*u1 - u1_{tt})/2 + u1/(2*\varepsilon) - (u + u1)/(2*\varepsilon**2)

h^2 =>


...

h^4 =>


-1097*\varepsilon**5*(u - u1)/120000 - 1097*\varepsilon**4*(3*u - 4*u1)/120000 - 1097*\varepsilon**3*(u - 2*u1)/30000 + 1097*\varepsilon**2*u1/15000 + 1097*\varepsilon*(u + 7*u1)/120000 + (3291*u + 500*u1_{ttttt})/60000 - 1097*(u - 7*u1)/(120000*\varepsilon) - 1097*u1/(15000*\varepsilon**2) + 1097*(u + 2*u1)/(30000*\varepsilon**3) - 1097*(3*u + 4*u1)/(120000*\varepsilon**4) + 1097*(u + u1)/(120000*\varepsilon**5)

h^5 =>


161*\varepsilon**6*(u - u1)/120000 + 161*\varepsilon**5*(4*u - 5*u1)/120000 + 161*\varepsilon**4*(7*u - 12*u1)/120000 + 161*\varepsilon**3*(u - 4*u1)/30000 - 161*\varepsilon**2*(u + 15*u1)/120000 - 161*\varepsilon*(8*u + 7*u1)/120000 - (966*u1 - 125*u1_{tttttt})/90000 - 161*(8*u - 7*u1)/(120000*\varepsilon) + 161*(u - 15*u1)/(120000*\varepsilon**2) + 161*(u + 4*u1)/(30000*\varepsilon**3) - 161*(7*u + 12*u1)/(120000*\varepsilon**4) + 161*(4*u + 5*u1)/(120000*\varepsilon**5) - 161*(u + u1)/(120000*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
\frac{\varepsilon^{2} \left(u - u_{1}\right)}{2} - \frac{\varepsilon u_{1}}{2} - \frac{2 u_{1} - u1_{tt}}{2} + \frac{u_{1}}{2 \varepsilon} - \frac{u + u_{1}}{2 \varepsilon^{2}}
\big)
+h^2\big(
- \frac{\varepsilon^{3} \left(u - u_{1}\right)}{6} - \frac{\varepsilon^{2} \left(u - 2 u_{1}\right)}{6} - \frac{\varepsilon \left(u - 3 u_{1}\right)}{6} + \frac{2 u + u1_{ttt}}{6} + \frac{u + 3 u_{1}}{6 \varepsilon} - \frac{u + 2 u_{1}}{6 \varepsilon^{2}} + \frac{u + u_{1}}{6 \varepsilon^{3}}
\big)
+h^3\big(
\frac{\varepsilon^{4} \left(u - u_{1}\right)}{24} + \frac{\varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{24} + \frac{\varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{24} - \frac{\varepsilon \left(2 u + 3 u_{1}\right)}{24} - \frac{4 u_{1} - u1_{tttt}}{24} - \frac{2 u - 3 u_{1}}{24 \varepsilon} - \frac{2 u + 5 u_{1}}{24 \varepsilon^{2}} + \frac{2 u + 3 u_{1}}{24 \varepsilon^{3}} - \frac{u + u_{1}}{24 \vare

In [40]:
f1 = NF(F1, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f1, eps)
prnlatex(f1, eps)

h^0 =>


-u1 + u_{t}

h^1 =>


0

h^2 =>


...

h^4 =>


97*\varepsilon**4*(u - u1)/120000 + 97*\varepsilon**3*(2*u - 3*u1)/120000 + 97*\varepsilon**2*(2*u - 5*u1)/120000 - 97*\varepsilon*(2*u + 3*u1)/120000 - 97*u1/30000 - 97*(2*u - 3*u1)/(120000*\varepsilon) - 97*(2*u + 5*u1)/(120000*\varepsilon**2) + 97*(2*u + 3*u1)/(120000*\varepsilon**3) - 97*(u + u1)/(120000*\varepsilon**4)

h^5 =>


77*\varepsilon**5*(u - u1)/90000 + 77*\varepsilon**4*(3*u - 4*u1)/90000 + 77*\varepsilon**3*(u - 2*u1)/22500 - 77*\varepsilon**2*u1/11250 - 77*\varepsilon*(u + 7*u1)/90000 - 77*u/15000 + 77*(u - 7*u1)/(90000*\varepsilon) + 77*u1/(11250*\varepsilon**2) - 77*(u + 2*u1)/(22500*\varepsilon**3) + 77*(3*u + 4*u1)/(90000*\varepsilon**4) - 77*(u + u1)/(90000*\varepsilon**5)

- u_{1} + u_{t}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
\frac{97 \varepsilon^{4} \left(u - u_{1}\right)}{120000} + \frac{97 \varepsilon^{3} \cdot \left(2 u - 3 u_{1}\right)}{120000} + \frac{97 \varepsilon^{2} \cdot \left(2 u - 5 u_{1}\right)}{120000} - \frac{97 \varepsilon \left(2 u + 3 u_{1}\right)}{120000} - \frac{97 u_{1}}{30000} - \frac{97 \cdot \left(2 u - 3 u_{1}\right)}{120000 \varepsilon} - \frac{97 \cdot \left(2 u + 5 u_{1}\right)}{120000 \varepsilon^{2}} + \frac{97 \cdot \left(2 u + 3 u_{1}\right)}{120000 \varepsilon^{3}} - \frac{97 \left(u + u_{1}\right)}{120000 \varepsilon^{4}}
\big)
+h^5\big(
\frac{77 \varepsilon^{5} \left(u - u_{1}\right)}{90000} + \frac{77 \varepsilon^{4} \cdot \left(3 u - 4 u_{1}\right)}{90000} + \frac{77 \varepsilon^{3} \left(u - 2 u_{1}\right)}{22500} - \frac{77 \varepsilon^{2} u_{1}}{11250} - \frac{77 \varepsilon \left(u + 7 u_{1}\right)}{90000} - \frac{77 u}{15000} + \frac{77 \left(u - 7 u_{1}\right)}{90000 \varepsilon} + \frac

In [41]:
f2 = NF(F2, [u.diff(t), u1.diff(t)], [F1, F2], head=False)
prn(f2, eps)
prnlatex(f2, eps)

h^0 =>


\varepsilon*(-u + u1) + u1_{t} + (u + u1)/\varepsilon

h^1 =>


0

h^2 =>


...

h^4 =>


-97*\varepsilon**5*(u - u1)/120000 - 97*\varepsilon**4*(3*u - 4*u1)/120000 - 97*\varepsilon**3*(u - 2*u1)/30000 + 97*\varepsilon**2*u1/15000 + 97*\varepsilon*(u + 7*u1)/120000 + 97*u/20000 - 97*(u - 7*u1)/(120000*\varepsilon) - 97*u1/(15000*\varepsilon**2) + 97*(u + 2*u1)/(30000*\varepsilon**3) - 97*(3*u + 4*u1)/(120000*\varepsilon**4) + 97*(u + u1)/(120000*\varepsilon**5)

h^5 =>


-77*\varepsilon**6*(u - u1)/90000 - 77*\varepsilon**5*(4*u - 5*u1)/90000 - 77*\varepsilon**4*(7*u - 12*u1)/90000 - 77*\varepsilon**3*(u - 4*u1)/22500 + 77*\varepsilon**2*(u + 15*u1)/90000 + 77*\varepsilon*(8*u + 7*u1)/90000 + 77*u1/11250 + 77*(8*u - 7*u1)/(90000*\varepsilon) - 77*(u - 15*u1)/(90000*\varepsilon**2) - 77*(u + 4*u1)/(22500*\varepsilon**3) + 77*(7*u + 12*u1)/(90000*\varepsilon**4) - 77*(4*u + 5*u1)/(90000*\varepsilon**5) + 77*(u + u1)/(90000*\varepsilon**6)

\varepsilon \left(- u + u_{1}\right) + u1_{t} + \frac{u + u_{1}}{\varepsilon}
+h\big(
0
\big)
+h^2\big(
0
\big)
+h^3\big(
0
\big)
+h^4\big(
- \frac{97 \varepsilon^{5} \left(u - u_{1}\right)}{120000} - \frac{97 \varepsilon^{4} \cdot \left(3 u - 4 u_{1}\right)}{120000} - \frac{97 \varepsilon^{3} \left(u - 2 u_{1}\right)}{30000} + \frac{97 \varepsilon^{2} u_{1}}{15000} + \frac{97 \varepsilon \left(u + 7 u_{1}\right)}{120000} + \frac{97 u}{20000} - \frac{97 \left(u - 7 u_{1}\right)}{120000 \varepsilon} - \frac{97 u_{1}}{15000 \varepsilon^{2}} + \frac{97 \left(u + 2 u_{1}\right)}{30000 \varepsilon^{3}} - \frac{97 \cdot \left(3 u + 4 u_{1}\right)}{120000 \varepsilon^{4}} + \frac{97 \left(u + u_{1}\right)}{120000 \varepsilon^{5}}
\big)
+h^5\big(
- \frac{77 \varepsilon^{6} \left(u - u_{1}\right)}{90000} - \frac{77 \varepsilon^{5} \cdot \left(4 u - 5 u_{1}\right)}{90000} - \frac{77 \varepsilon^{4} \cdot \left(7 u - 12 u_{1}\right)}{90000} - \frac{77 \varepsilon^{3} \left(u - 4 u_{1}\right)}{225